In [32]:
import random
import sc2
from sc2 import run_game, maps, Race, Difficulty
from sc2.player import Bot, Computer
from sc2.constants import NEXUS, PROBE, PYLON, ASSIMILATOR, GATEWAY, CYBERNETICSCORE, STALKER, STARGATE, VOIDRAY

from examples.terran.proxy_rax import ProxyRaxBot

class SentdeBot(sc2.BotAI):
    def __init__(self):
        self.IT_PER_MIN = 165
        self.MAX_WORKERS = 50
    
    async def on_step(self, iteration):
        # what to do every step
        self.iteration = iteration
        await self.distribute_workers()  # in sc2/bot_ai.py
        await self.build_workers()
        await self.build_pylons() #supply building
        await self.expand()
        await self.build_assimilator() #getting gas
        #offensive/defensive forces
        await self.offensive_force_buildings()
        await self.build_offensive_force()
        await self.attack()

    async def build_workers(self):
        #nexus = command center
        if (len(self.units(NEXUS)) * 16) > len(self.units(PROBE)) and\
            len(self.units(PROBE)) < self.MAX_WORKERS:
            for nexus in self.units(NEXUS).ready.noqueue:
                if self.can_afford(PROBE):
                    await self.do(nexus.train(PROBE))
    
    async def build_pylons(self):
        if self.supply_left < 5 and \
           not self.already_pending(PYLON):
            nexuses = self.units(NEXUS).ready
            if nexuses.exists:
                if self.can_afford(PYLON):
                    await self.build(PYLON, near=nexuses.first)
    
    async def expand(self):
        if self.units(NEXUS).amount < (self.iteration / self.IT_PER_MIN) and \
           self.can_afford(NEXUS):
            await self.expand_now()
    
    async def build_assimilator(self):
        for nexus in self.units(NEXUS).ready:
            vaspenes = self.state.vespene_geyser.closer_than(15.0, nexus)
            for vaspene in vaspenes:
                if not self.can_afford(ASSIMILATOR):
                    break
                worker = self.select_build_worker(vaspene.position)
                if worker is None:
                    break
                if not self.units(ASSIMILATOR).closer_than(1.0, vaspene).exists:
                    await self.do(worker.build(ASSIMILATOR, vaspene))
                    
    async def offensive_force_buildings(self):
        #print(self.iteration / self.IT_PER_MIN)
        if self.units(PYLON).ready.exists:
            pylon = self.units(PYLON).ready.random

            if self.units(GATEWAY).ready.exists and not self.units(CYBERNETICSCORE):
                if self.can_afford(CYBERNETICSCORE) and not self.already_pending(CYBERNETICSCORE):
                    await self.build(CYBERNETICSCORE, near=pylon)

            elif len(self.units(GATEWAY)) < ((self.iteration / self.IT_PER_MIN)/2):
                if self.can_afford(GATEWAY) and not self.already_pending(GATEWAY):
                    await self.build(GATEWAY, near=pylon)
            
            if self.units(CYBERNETICSCORE).ready.exists:
                if len(self.units(STARGATE)) < ((self.iteration / self.IT_PER_MIN)/2):
                    if self.can_afford(STARGATE) and not self.already_pending(STARGATE):
                        await self.build(STARGATE, near=pylon)
    
    async def build_offensive_force(self):
        for gw in self.units(GATEWAY).ready.noqueue:
            if not self.units(STALKER).amount > self.units(VOIDRAY).amount:
                if self.units(CYBERNETICSCORE).ready and \
                   self.can_afford(STALKER) and \
                   self.supply_left > 0:
                    await self.do(gw.train(STALKER))
        
        for sg in self.units(STARGATE).ready.noqueue:
            if self.can_afford(VOIDRAY) and self.supply_left > 0:
                await self.do(sg.train(VOIDRAY))
    
    def find_target(self, state):
        if len(self.known_enemy_units) > 0:
            return random.choice(self.known_enemy_units)
        elif len(self.known_enemy_structures) > 0:
            return random.choice(self.known_enemy_structures)
        else:
            return self.enemy_start_locations[0]
    
    async def attack(self):
        #{UNIT: [n to fight, n to defend]}
        aggressive_units = {STALKER: [15, 5],
                           VOIDRAY: [ 8, 3]}
        
        for UNIT in aggressive_units:
            if self.units(UNIT).amount > aggressive_units[UNIT][0] and\
               self.units(UNIT).amount > aggressive_units[UNIT][1]:
                for s in self.units(UNIT).idle:
                    await self.do(s.attack(self.find_target(self.state)))
            if self.units(UNIT).amount > aggressive_units[UNIT][1]:
                if len(self.known_enemy_units) > 0:
                    for s in self.units(UNIT).idle:
                        await self.do(s.attack(random.choice(self.known_enemy_units)))
    
        
run_game(maps.get("AbyssalReefLE"), [
    Bot(Race.Protoss, SentdeBot()),
    Bot(Race.Terran, ProxyRaxBot())
    #Computer(Race.Terran, Difficulty.Hard)
], realtime=False)

INFO:sc2.protocol:Client status changed to Status.launched (was None)
INFO:sc2.controller:Creating new game
INFO:sc2.controller:Map:     AbyssalReefLE
INFO:sc2.controller:Players: Bot(Race.Protoss, <__main__.SentdeBot object at 0x000002468A4F68D0>), Bot(Race.Terran, <examples.terran.proxy_rax.ProxyRaxBot object at 0x000002468C399390>)
INFO:sc2.protocol:Client status changed to Status.init_game (was Status.launched)
INFO:sc2.protocol:Client status changed to Status.launched (was None)
INFO:sc2.protocol:Client status changed to Status.in_game (was None)
INFO:root:Player id: 2
INFO:sc2.protocol:Client status changed to Status.in_game (was None)
INFO:root:Player id: 1
ERROR:sc2.bot_ai:Error: ActionResult.NotSupported (action: UnitCommand(AbilityId.PROTOSSBUILD_GATEWAY, Unit(name='Probe', tag=4368629762), (70, 120), False))
INFO:sc2.protocol:Client status changed to Status.ended (was Status.in_game)
INFO:sc2.protocol:Client status changed to Status.ended (was Status.in_game)
INFO:root:Resul

[None, <Result.Victory: 1>]